#### Инструкция
Загрузите данные, приведите их к числовым, заполните пропуски, нормализуйте данные и оптимизируйте память.

Сформируйте параллельный ансамбль из CatBoost, градиентного бустинга, XGBoost и LightGBM. Используйте лучшие гиперпараметры, подобранные ранее, или найдите их через перекрестную проверку. Итоговое решение рассчитайте на основании самого точного предсказания класса у определенной модели ансамбля: выберите для каждого класса модель, которая предсказывает его лучше всего.

Проведите расчеты и выгрузите результат в виде submission.csv

Данные:
* video.ittensive.com/machine-learning/prudential/train.csv.gz
* video.ittensive.com/machine-learning/prudential/test.csv.gz
* video.ittensive.com/machine-learning/prudential/sample_submission.csv.gz

Итоговый файл с кодом (.py или .ipynb) выложите в github с портфолио.

#### Задача
Загрузите данные, приведите их к числовым, заполните пропуски, нормализуйте данные и оптимизируйте память.

### Подключение библиотек и функции оптимизации памяти

In [1]:
import numpy as np 
import pandas as pd

# библиотеки для ансамбля 
from catboost import Pool, CatBoostClassifier  # CatBoost
from sklearn.ensemble import GradientBoostingClassifier  # Градиентный бустинг
from xgboost import XGBClassifier  # XGBoost
import lightgbm as lgb  # LightGBM

from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from sklearn import preprocessing


def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

Загрузка данных, приведение их к числовым, заполнение пропусков, нормализация данных и оптимизация памяти.

### Загрузка данных

In [2]:
# data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
data = pd.read_csv("train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


### Предобработка данных
Создадим отдельную функцию для предобработки данных. Её нужно будет применить как к обучающим, так и к тестовым данным. 

In [3]:
def data_preprocess (df):
    df["Product_Info_2_1"] = df["Product_Info_2"].str.slice(0, 1)
    df["Product_Info_2_2"] = pd.to_numeric(df["Product_Info_2"].str.slice(1, 2))
    df.drop(labels=["Product_Info_2"], axis=1, inplace=True)
    for l in df["Product_Info_2_1"].unique():
        df["Product_Info_2_1" + l] = df["Product_Info_2_1"].isin([l]).astype("int8")
    df.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)
    df.fillna(value=-1, inplace=True)
    return df
# вывели data["Response"] = data["Response"] - 1 из  под функции, так как портятся данные при в data при каждом применении функции.
data["Response"] = data["Response"] - 1  # уменьшим класс на 1.


Проведем предобработку тренировочных данных.   

In [4]:
data = data_preprocess(data)

### Набор столбцов для расчета
Создадим список столбцов для расчета в модели (вкл. исправление опечатки в InsuredInfo)

In [5]:
columns_groups = ["Insurance_History", "InsuredInfo", "Medical_Keyword",
                  "Family_Hist", "Medical_History", "Product_Info"]
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'InsuredInfo_1', 'InsuredInfo_2', 'InsuredInfo_3', 'InsuredInfo_4', 'InsuredInfo_5', 'InsuredInfo_6', 'InsuredInfo_7', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keywor

### Нормализация данных 

In [6]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data,
                                                     columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

### Оптимизация памяти
Воспользуемся функцией оптимизации памяти для сформированных данных.

In [7]:
data_transformed = reduce_mem_usage(data_transformed)
print (data_transformed.info())
# print (data_transformed)

Потребление памяти меньше на 42.87 Мб (минус 75.1 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 126 entries, 0 to Response
dtypes: float16(125), int8(1)
memory usage: 14.2 MB
None


### Задача
Сформируйте параллельный ансамбль из CatBoost, градиентного бустинга, XGBoost и LightGBM. Используйте лучшие гиперпараметры, подобранные ранее, или найдите их через перекрестную проверку. Итоговое решение рассчитайте на основании самого точного предсказания класса у определенной модели ансамбля: выберите для каждого класса модель, которая предсказывает его лучше всего.

### Построение базовых моделей
Сформируем параллельный ансамбль из CatBoost, градиентного бустинга, XGBoost и LightGBM. Использовать будем лучшие гиперпараметры, который были подобраны ранее. 

In [8]:
x = pd.DataFrame(data_transformed, columns=columns_transformed)

__CatBoost.__ Оптимальные парамеры которые нашли ранее - 'depth': 7, 'l2_leaf_reg': 1, 'learning_rate': 0.5700000000000001

In [9]:
model_cat_boost = CatBoostClassifier(iterations=1000,
        learning_rate=0.5700000000000001,
        depth=7,
        l2_leaf_reg=1,
        random_seed=17, loss_function="MultiClass",
        bootstrap_type="MVS", custom_metric="WKappa")
model_cat_boost.fit(Pool(data=x, label=data["Response"]))

0:	learn: 1.5279394	total: 348ms	remaining: 5m 47s
1:	learn: 1.4322561	total: 554ms	remaining: 4m 36s
2:	learn: 1.3338654	total: 763ms	remaining: 4m 13s
3:	learn: 1.2977580	total: 964ms	remaining: 3m 59s
4:	learn: 1.2739421	total: 1.17s	remaining: 3m 51s
5:	learn: 1.2594011	total: 1.37s	remaining: 3m 47s
6:	learn: 1.2479667	total: 1.57s	remaining: 3m 42s
7:	learn: 1.2403408	total: 1.75s	remaining: 3m 37s
8:	learn: 1.2278383	total: 1.95s	remaining: 3m 34s
9:	learn: 1.2176685	total: 2.15s	remaining: 3m 32s
10:	learn: 1.2127665	total: 2.34s	remaining: 3m 30s
11:	learn: 1.2079266	total: 2.53s	remaining: 3m 28s
12:	learn: 1.1996008	total: 2.73s	remaining: 3m 27s
13:	learn: 1.1917714	total: 2.92s	remaining: 3m 25s
14:	learn: 1.1835361	total: 3.11s	remaining: 3m 24s
15:	learn: 1.1794947	total: 3.33s	remaining: 3m 24s
16:	learn: 1.1758466	total: 3.54s	remaining: 3m 24s
17:	learn: 1.1729276	total: 3.73s	remaining: 3m 23s
18:	learn: 1.1601366	total: 3.93s	remaining: 3m 23s
19:	learn: 1.1567989	t

__Градиентный бустинг.__ Оптимальные парамеры которые нашли ранее - 'max_depth': 15, 'max_features': 26, 'min_samples_leaf': 20, 'n_estimators': 75

In [10]:
# параметры от случайного леса: max_depth=17, max_features=27, min_samples_leaf=20, n_estimators=76.

model_gb = GradientBoostingClassifier(random_state=17,
        min_samples_leaf=20,
        max_features=26,
        max_depth=15,
        n_estimators=75)
model_gb.fit(x, data["Response"])

GradientBoostingClassifier(max_depth=15, max_features=26, min_samples_leaf=20,
                           n_estimators=75, random_state=17)

__XGBoost.__ Параметры от случайного леса: max_depth=17, max_features=27, min_samples_leaf=20, n_estimators=76

In [11]:
model_xgb = XGBClassifier(max_depth=17, 
            max_features=27,
            n_estimators=76, 
            min_samples_leaf=20)
model_xgb.fit(x, data["Response"])

[15:39:49] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-030221e36e1a46bfb-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "max_features", "min_samples_leaf" } are not used.



XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=17, max_features=27, max_leaves=0,
              min_child_weight=1, min_samples_leaf=20, missing=nan,
              monotone_constraints='()', n_estimators=76, n_jobs=0,
              num_parallel_tree=1, ...)

__LightGBM.__ Оптимальные парамеры которые нашли ранее - 'max_depth': 17, 'min_child_samples': 18, 'num_leaves': 34

In [12]:
model_lgb = lgb.LGBMRegressor(random_state=17, max_depth=18,
            min_child_samples=18, num_leaves=75, n_estimators=1000)
model_lgb.fit(x, data["Response"])

LGBMRegressor(max_depth=18, min_child_samples=18, n_estimators=1000,
              num_leaves=75, random_state=17)

#### Определение лучших моделей предсказания каждого класса

In [13]:
data_train = data_transformed.copy()
x= pd.DataFrame(data_train, columns=columns_transformed)
data_train["target_cat_boost"] = model_cat_boost.predict(Pool(data=x))
# data_train["target_cat_boost"] = model_cat_boost.predict(Pool(data=x, label=data_train["Response"]))
data_train["target_gb"] = model_gb.predict(x)
data_train["target_xgb"] = model_xgb.predict(x)
data_train["target_lgb"] = np.round(model_lgb.predict(x)).astype("int8")


In [14]:
print(data_train.head())

          0         1         2         3         4         5         6  \
0 -1.618164 -1.690430  1.198242 -1.198242 -1.634766 -0.169434  0.862305   
1 -1.805664 -1.445312 -1.753906 -1.613281  0.611816 -0.169434  0.862305   
2 -0.043610  0.514160 -1.905273 -0.332764  0.611816 -0.169434 -1.159180   
3 -0.983398 -0.465576 -1.224609 -0.957520  0.611816 -0.169434 -1.159180   
4 -0.654297 -0.710449  0.062622 -0.371582  0.611816 -0.169434 -1.159180   

          7         8         9  ...       120       121       122       123  \
0 -1.013672  0.862305 -0.928711  ...  0.750977 -0.623535 -0.215942 -0.128906   
1 -1.013672  0.861328 -0.928711  ... -1.332031  1.604492 -0.215942 -0.128906   
2  1.100586 -1.156250  1.130859  ... -1.332031 -0.623535  4.628906 -0.128906   
3  1.100586 -1.156250  1.130859  ...  0.750977 -0.623535 -0.215942 -0.128906   
4  1.100586 -1.156250  1.130859  ...  0.750977 -0.623535 -0.215942 -0.128906   

       124  Response  target_cat_boost  target_gb  target_xgb  targe

In [16]:
print ("Результат cat_boost:",
       round(cohen_kappa_score(data_train["target_cat_boost"],
                    data["Response"], weights='quadratic'), 3))
print (confusion_matrix(data_train["target_cat_boost"], data["Response"]))

Результат cat_boost: 0.958
[[ 5841    10     0     0     4     3     2     3]
 [   10  6126     0     0     5    15     6     1]
 [    1     0  1013     0     0     0     0     0]
 [    2     2     0  1420     0     0     0     0]
 [   17    36     0     0  5218    22     1     2]
 [   83   116     0     1    85 10574    97    77]
 [   56    51     0     0    47   143  7314    89]
 [  197   211     0     7    73   476   607 19317]]


In [17]:
print ("Результат gb:",
       round(cohen_kappa_score(data_train["target_gb"],
                    data_train["Response"], weights='quadratic'), 3))
print (confusion_matrix(data_train["target_gb"], data_train["Response"]))

Результат gb: 0.883
[[ 5066     9     0     0     1     5     2     1]
 [   14  5506     0     0     4    11     1     1]
 [    1     3  1013     0     0     0     0     0]
 [   12    11     0  1428     0     0     0     0]
 [   47    38     0     0  4991    23     1     5]
 [  290   279     0     0   109  9989    97    84]
 [  163   155     0     0    99   212  6938    36]
 [  614   551     0     0   228   993   988 19362]]


In [18]:
print ("Результат xgb:",
       round(cohen_kappa_score(data_train["target_xgb"],
                    data_train["Response"], weights='quadratic'), 3))
print (confusion_matrix(data_train["target_xgb"], data_train["Response"]))

Результат xgb: 0.998
[[ 6181     0     0     0     0     0     0     0]
 [    0  6538     0     0     0     0     0     0]
 [    0     0  1013     0     0     0     0     0]
 [    0     0     0  1428     0     0     0     0]
 [    0     0     0     0  5432     0     0     0]
 [    1     0     0     0     0 11226     0     0]
 [    4     0     0     0     0     0  8000     0]
 [   21    14     0     0     0     7    27 19489]]


In [19]:
print ("Результат lgb:",
       round(cohen_kappa_score(data_train["target_lgb"],
                    data_train["Response"], weights='quadratic'), 3))
print (confusion_matrix(data_train["target_lgb"], data_train["Response"]))

Результат lgb: 0.894
[[    0    25     0     0     0     0     0     0     0     0]
 [    0  1634    85     0     0     0     0     0     0     0]
 [    0  2225  2366    77     5     1     1     0     0     0]
 [    0  1220  2319   704   327    77     7     0     0     0]
 [    0   613  1015   166   854  1345   239    13     2     0]
 [    0   337   516    47   105  3242  3090   223    38     0]
 [    0   131   204    17    88   649  6622  2862   684     0]
 [    0    22    46     2    49   118  1222  4570  7944     0]
 [    0     0     1     0     0     0    52   359 10796     0]
 [    0     0     0     0     0     0     0     0    25     0]]


Судя по матрицам неточности XGBoost лучше всего справляется с предсказаниями, а также с классом 2 и 3 так-же хорошо справился Градиентный бустинг, возможно это обоснованно подобранными параметрами моделей. 

Если в CatBoost количество итераций увеличить до 10000, то он станет самой точной моделью с точностью  1.

Попробуем произвести перекрестную проверку по всем данным. 

In [20]:
class_target = ["target_gb"]*8  # возьмем как отправной набор для моделей для каждой классификации, так как у нее самая низкая Каппа оценка.
def vote_class_enumerate (x):
    for _,target in enumerate(class_target):
        # print("t1", target)
        if x[target] == _:
            # print("t2", x[target], _)
            x["Response"] = x[target]
            break
    return x

In [21]:
kappa_min = 0
for target_model in ["lgb", "cat_boost", "gb", "xgb"]:
    print ("Проверяем модель:", target_model)
    target_model = "target_" + target_model  # берем каждую модель по списку
    for class_ in range(0,8):
        target_model_prev = class_target[class_]  # модель из списка class_target
        # print("1", target_model_prev)
        class_target[class_] = target_model  # меняем на target_model 
        # print("2",  class_target[class_])
        data_train = data_train.apply(vote_class_enumerate, axis=1)  # формируем Response взяв номер класса предсказанный проверяемой моделью
        kappa = cohen_kappa_score(data_train["Response"],  # Оцениваем по Kappa точность предсказаний. 
                data["Response"], weights='quadratic')
        # print(data_train["Response"].head(15))
        # print(data_train["Response"].head(15))
        # print("3", kappa)
        if kappa > kappa_min: # если точность повысилась обновляем kappa_min до вычисленной kappa с этим списком моделей предсказания для каждого класса
            kappa_min = kappa
            # print('4', kappa_min)
        else:
            class_target[class_] = target_model_prev # если нет, возвращаем как было.
            # print('5', target_model_prev)
        # print ('6', class_target)
    print ("Максимальная оценка:", kappa_min)
print (class_target)

Проверяем модель: lgb
Максимальная оценка: 0.9068297975031382
Проверяем модель: cat_boost
Максимальная оценка: 0.9583805759729961
Проверяем модель: gb
Максимальная оценка: 0.9589260109192023
Проверяем модель: xgb
Максимальная оценка: 0.9975475840956972
['target_xgb', 'target_xgb', 'target_gb', 'target_xgb', 'target_xgb', 'target_xgb', 'target_xgb', 'target_xgb']


In [22]:
print(data_train.head())

          0         1         2         3         4         5         6  \
0 -1.618164 -1.690430  1.198242 -1.198242 -1.634766 -0.169434  0.862305   
1 -1.805664 -1.445312 -1.753906 -1.613281  0.611816 -0.169434  0.862305   
2 -0.043610  0.514160 -1.905273 -0.332764  0.611816 -0.169434 -1.159180   
3 -0.983398 -0.465576 -1.224609 -0.957520  0.611816 -0.169434 -1.159180   
4 -0.654297 -0.710449  0.062622 -0.371582  0.611816 -0.169434 -1.159180   

          7         8         9  ...       120       121       122       123  \
0 -1.013672  0.862305 -0.928711  ...  0.750977 -0.623535 -0.215942 -0.128906   
1 -1.013672  0.861328 -0.928711  ... -1.332031  1.604492 -0.215942 -0.128906   
2  1.100586 -1.156250  1.130859  ... -1.332031 -0.623535  4.628906 -0.128906   
3  1.100586 -1.156250  1.130859  ...  0.750977 -0.623535 -0.215942 -0.128906   
4  1.100586 -1.156250  1.130859  ...  0.750977 -0.623535 -0.215942 -0.128906   

       124  Response  target_cat_boost  target_gb  target_xgb  targe

In [23]:
print (class_target)

['target_xgb', 'target_xgb', 'target_gb', 'target_xgb', 'target_xgb', 'target_xgb', 'target_xgb', 'target_xgb']


Перекрестная проверка подтверждает выводы сделанные на основе матриц неточности.

Судя по перекрестной проверке, для всех классов, кроме 2го, точнее работает XGBoost, для 2 - Градиентный бустинг.

Будим использовать это при формировании финального результата. 

In [24]:
def vote_class_test (x):
    if x.target_gb == 2:  # если gb предсказал 2 класс, берем его, так как этот класс gb предсказывает лучше всех
        class_ = x.target_gb
    # elif x.target_gb == 3:
    #     class_ = x.target_gb
    else:
        class_ = x.target_xgb  # для всего остального берем xgb 
    x["Target"] = class_
    return x

Проведем предсказания и проверку для оценки качества ансамбля моделей.

In [25]:
data_train = data_train.apply(vote_class_test, axis=1)
print (data_train.head(15))

           0         1         2         3         4         5         6  \
0  -1.618164 -1.690430  1.198242 -1.198242 -1.634766 -0.169434  0.862305   
1  -1.805664 -1.445312 -1.753906 -1.613281  0.611816 -0.169434  0.862305   
2  -0.043610  0.514160 -1.905273 -0.332764  0.611816 -0.169434 -1.159180   
3  -0.983398 -0.465576 -1.224609 -0.957520  0.611816 -0.169434 -1.159180   
4  -0.654297 -0.710449  0.062622 -0.371582  0.611816 -0.169434 -1.159180   
5   0.073853  1.738281  0.516602 -0.855469  0.611816 -0.169434  0.862305   
6  -1.335938 -1.690430 -0.164429 -0.759766 -1.634766 -0.169434  0.862305   
7   1.249023  1.003906  1.046875  0.835938 -1.634766 -0.169434  0.862305   
8  -1.218750 -1.200195  0.592285 -0.874023 -1.634766 -0.169434  0.862305   
9  -0.090637 -1.445312  0.743652  0.968262  0.611816 -0.169434  0.862305   
10  0.191284 -0.220581  0.667969  0.427246 -1.634766 -0.169434  0.862305   
11 -0.231567 -0.220581 -0.542969 -0.117920  0.611816 -0.169434 -1.159180   
12 -1.453125

Оценим по Каппа метрике и выведем результат матрицы неточностей

In [26]:
print ("Результат:",
       round(cohen_kappa_score(data_train["Target"],
                    data["Response"], weights='quadratic'), 3))
print (confusion_matrix(data_train["Target"], data["Response"]))

Результат: 0.998
[[ 6180     0     0     0     0     0     0     0]
 [    0  6535     0     0     0     0     0     0]
 [    1     3  1013     0     0     0     0     0]
 [    0     0     0  1428     0     0     0     0]
 [    0     0     0     0  5432     0     0     0]
 [    1     0     0     0     0 11226     0     0]
 [    4     0     0     0     0     0  8000     0]
 [   21    14     0     0     0     7    27 19489]]


In [27]:
print (class_target)

['target_xgb', 'target_xgb', 'target_gb', 'target_xgb', 'target_xgb', 'target_xgb', 'target_xgb', 'target_xgb']


In [28]:
def vote_class (x):
    if x.target_gb == 2:  # если gb предсказал 2 класс, берем его, так как этот класс gb предсказывает лучше всех
        class_ = x.target_gb
    # elif x.target_gb == 3: 
    #     class_ = x.target_gb
    else:
        class_ = x.target_xgb  # для всего остального берем xgb 
    x["Response"] = class_ + 1
    return x

### Загрузка данных для расчетов
Применим построенный ансамбль моделей для расчета актуальных данных (актуал. значения классификации).

Будем использовать ранее написанные функции для нормализация и оптимизации данных.

In [29]:
# data_test = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/test.csv.gz")
data_test = pd.read_csv("test.csv.gz")
data_test = data_preprocess(data_test)
data_test = reduce_mem_usage(data_test)
data_test_transformed = pd.DataFrame(scaler.transform(pd.DataFrame(data_test,
                                            columns=columns)))
print (data_test_transformed.info())

Потребление памяти меньше на 16.34 Мб (минус 84.9 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19765 entries, 0 to 19764
Columns: 125 entries, 0 to 124
dtypes: float32(125)
memory usage: 9.4 MB
None


In [30]:
print(data_test_transformed.head())

        0         1         2         3         4         5         6    \
0  0.519789  1.002921  1.045952  0.022140  0.611857 -0.169414 -1.159587   
1  0.215425  0.266273  1.122714  0.126020 -1.634368 -0.169414  0.862391   
2  0.308653  0.022915  0.894903  0.405695  0.611857 -0.169414 -1.159587   
3 -0.278139 -0.707156  0.592804  0.143999 -1.634368 -0.169414  0.862391   
4 -0.513953 -0.463799 -0.542540 -0.333447  0.611857 -0.169414 -1.159587   

        7         8         9    ...       115       116       117       118  \
0  1.101046 -1.156735  1.130555  ...  0.559558 -0.083689  0.441621 -0.149284   
1 -1.013721  0.864261 -0.928723  ... -0.892015 -0.083689  0.441621 -0.149284   
2  1.101046 -1.156735  1.130555  ... -0.652247 -0.083689  0.441621 -0.149284   
3 -1.013721  0.862242  0.100916  ... -0.627190 -0.083689 -2.264385 -0.149284   
4  1.101046 -1.156735  1.130555  ... -0.892015 -0.083689  0.441621 -0.149284   

        119       120       121       122       123       124  
0 -0

### Расчет предсказаний
Произведем предсказания класса по ранее построенному ансамблю 

In [31]:
data_test["target_xgb"] = model_xgb.predict(data_test_transformed)
data_test["target_cat_boost"] = model_cat_boost.predict(Pool(data=data_test_transformed))
data_test["target_gb"] = model_gb.predict(data_test_transformed)
data_test["target_lgb"] = np.round(model_lgb.predict(data_test_transformed)).astype("int8")

C:\Users\Alexa\AppData\Local\Temp\ipykernel_16648\1027456068.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_test["target_xgb"] = model_xgb.predict(data_test_transformed)
C:\Users\Alexa\AppData\Local\Temp\ipykernel_16648\1027456068.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_test["target_cat_boost"] = model_cat_boost.predict(Pool(data=data_test_transformed))
C:\Users\Alexa\AppData\Local\Temp\ipykernel_16648\1027456068.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of c

In [32]:
print(data_test.head())

   Id  Product_Info_1  Product_Info_3  Product_Info_4  Product_Info_5  \
0   1               1              26        0.487061               2   
1   3               1              26        0.076904               2   
2   4               1              26        0.144653               2   
3   9               1              26        0.151733               2   
4  12               1              26        0.076904               2   

   Product_Info_6  Product_Info_7   Ins_Age        Ht        Wt  ...  \
0               3               1  0.611816  0.781738  0.338867  ...   
1               3               1  0.626953  0.727051  0.311768  ...   
2               3               1  0.582031  0.708984  0.320068  ...   
3               1               1  0.522461  0.654785  0.267822  ...   
4               3               1  0.298584  0.672852  0.246826  ...   

   Product_Info_2_2  Product_Info_2_1D  Product_Info_2_1A  Product_Info_2_1E  \
0                 3                  1          

Сформируем предсказание Response, на основе ансамбля моделей.

In [33]:
data_test = data_test.apply(vote_class, axis=1)
print (data_test.head())

     Id  Product_Info_1  Product_Info_3  Product_Info_4  Product_Info_5  \
0   1.0             1.0            26.0        0.487061             2.0   
1   3.0             1.0            26.0        0.076904             2.0   
2   4.0             1.0            26.0        0.144653             2.0   
3   9.0             1.0            26.0        0.151733             2.0   
4  12.0             1.0            26.0        0.076904             2.0   

   Product_Info_6  Product_Info_7   Ins_Age        Ht        Wt  ...  \
0             3.0             1.0  0.611816  0.781738  0.338867  ...   
1             3.0             1.0  0.626953  0.727051  0.311768  ...   
2             3.0             1.0  0.582031  0.708984  0.320068  ...   
3             1.0             1.0  0.522461  0.654785  0.267822  ...   
4             3.0             1.0  0.298584  0.672852  0.246826  ...   

   Product_Info_2_1D  Product_Info_2_1A  Product_Info_2_1E  Product_Info_2_1B  \
0                1.0               

### Формирование результатов
Загрузим пример данных для отправки и заменим в нем столбец Response на рассчитанный ранее.

In [34]:
submission = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/sample_submission.csv.gz")
# submission = pd.read_csv("sample_submission.csv.gz")

print (submission.head())

   Id  Response
0   1         8
1   3         8
2   4         8
3   9         8
4  12         8


In [35]:
submission["Response"] = data_test["Response"].astype("int8")
print (submission.head())

   Id  Response
0   1         7
1   3         8
2   4         6
3   9         8
4  12         8


### Выгрузка результатов

In [36]:
submission.to_csv("submission.csv", index=False)